In [222]:
#region imports
import pandas as pd
import numpy as np
import missingno as msno
import random
import re
from nltk import word_tokenize
import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from tensorflow.keras import Sequential,layers
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display="diagram")

from sklearn.model_selection import cross_val_score,KFold,train_test_split
from xgboost import XGBRegressor
import lightgbm as lgbm
from sklearn.ensemble import BaggingRegressor

rand_state=100
RANDOMSEED = 100
DISPLAY_WIDTH = 400
DISPLAYMAX_COLUMNS = 25
#endregion

#region settings
random.seed(RANDOMSEED)
pd.set_option('display.width', DISPLAY_WIDTH)
pd.set_option('display.max_columns', DISPLAYMAX_COLUMNS)
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

#endregion

In [223]:
df_awards=pd.read_csv("../data/220k_awards_by_directors.csv")
df_500_fav=pd.read_csv("../data/500 favorite directors_with wikipedia summary.csv", sep=";",header=0,names=['A', 'B'])
df_900_acc=pd.read_csv("../data/900_acclaimed_directors_awards.csv", sep=";")
df_all_casting=pd.read_csv("../data/AllMoviesCastingRaw.csv", sep=";")
df_all_details = pd.read_csv("../data/AllMoviesDetailsCleaned.csv", sep=";")
df_lan2country_df = pd.read_csv('../data/language to country.csv')
df_most_common_lang=pd.read_csv("../data/MostCommonLanguageByDirector.csv")
df_spliberg=pd.read_csv("../data/spielberg_awards.csv",encoding='latin1')

/tmp/ipykernel_23433/3009648048.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all_details = pd.read_csv("../data/AllMoviesDetailsCleaned.csv", sep=";")


In [224]:
def show(df_, rows=20, cols=30, title=None):
    '''
    this utility function, can be passed in a 
    panda's processing chain, to display intermediate
    dataframes.
    '''
    if title:
        display(HTML(f'<h2 >{title}</h2 > '))
        with pd.option_context('display.min_rows', rows, 'display.max_columns', cols):
            display(df_)
    return df_

def get_var(df, var_name):
    '''
    this utility function, can be passed in a 
    panda's processing chain, to store an intermediate
    dataframes into var_names.
    '''
    globals()[var_name] = df
    return df

def tokenize(text):
    '''this method does the following
    1. normalizing all the words to lower size
    2. removes punctuations
    3. splits the words
    4. removes the stopwords like am,is,have,you,...
    5. lammetizes the words for example running-->run
    '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())    # normalize case and remove punctuation
    tokens = word_tokenize(text)    # tokenize text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    # lemmatize andremove stop words
    return tokens

def prep_data(text,method=CountVectorizer):
    '''
    this method counts either counts the words 
    in sentences (CountVectorizer) or wights them 
    based on their importance in the sentence 
    and entire data(TfidfVectorizer):
    '''
    count_vector = method(tokenizer=tokenize)
    count_vector.fit(text)
    doc_array = count_vector.transform(text).toarray()
    frequency_matrix_count = pd.DataFrame(doc_array, columns=count_vector.get_feature_names_out())
    return frequency_matrix_count



In [225]:
def tweak_details(df_all_details):
    return (#1- major cleanups
    df_all_details  
        .query("status in ['Released']")
        .pipe(lambda df_:df_.replace("none",np.NaN))
        .assign(revenue=lambda df_:df_.revenue.replace(0,np.NAN),
            genres=lambda df_:df_.genres.fillna("none"),
            runtime=lambda df_:df_.runtime.fillna(-999),
            # original_language=lambda df_:df_.original_language.fillna("none"),
            day_of_week_temp=lambda df_:pd.to_datetime(df_.release_date,dayfirst=True),
            day_of_week=lambda f_:f_.day_of_week_temp.apply( lambda d:d.weekday()),
            year=lambda df_:pd.to_datetime(df_.release_date,dayfirst=True).dt.year,
            age=lambda df_:(2025-pd.to_datetime(df_.release_date,dayfirst=True).dt.year).fillna(-999),
            month=lambda df_:(pd.to_datetime(df_.release_date,dayfirst=True).dt.month),
            sin_month=lambda df_:(np.sin(2*np.pi*(df_.month-1)/12)).fillna(-999),
            cos_month=lambda df_:(np.cos(2*np.pi*(df_.month-1)/12)).fillna(-999),
            popularity=lambda df_:df_.popularity.astype("str").apply(lambda x:x.replace(",","")).astype("float") ,
            original_language=lambda df_:df_.original_language.apply(lambda x:1 if x=="en" else 0),
            production_countries=lambda df_:df_.production_countries.apply(lambda x:1 if x=="United States of America" else 0),
            spoken_languages=lambda df_:df_.spoken_languages.apply(lambda x:1 if x=="English" else 0),
      
            )
        
        .dropna(subset="revenue")
        .query("revenue > 0")
        .reset_index(drop=True)
        .pipe(lambda df_:pd.concat([df_,
                                prep_data(df_.genres)],axis=1))
        .drop(columns=["imdb_id","original_title",
                    "overview","status","tagline","title","vote_average","vote_count",
                    "production_companies","release_date","day_of_week_temp","month"])
    
    )
    
def fame_func(data_awards_cleaned,director,dt_,stat=0):

    try: 
        if stat==0:
            return data_awards_cleaned.loc[director][data_awards_cleaned.loc[director].index<=dt_].tail(1).nominated_cumsum.values[0]
        else:
            return data_awards_cleaned.loc[director][data_awards_cleaned.loc[director].index<=dt_].tail(1).won_cumsum.values[0]
            
    except:
        return np.NaN

def tweak_cast(df_all_casting,df):
    return(#2- extracting actor_weights
            df_all_casting 
                .melt(id_vars="id",value_vars=["actor1_name","actor2_name","actor3_name","actor4_name","actor5_name"])
                .sort_values(by=["id","variable"])
                .merge(df[["id","year"]],on="id",how="left")
                .replace("none",np.NaN)
                .dropna()
                .assign(unit=1)
                .sort_values(by=["value","year"])
                .assign(actor_freq=lambda df_:df_.groupby("value").cumsum()["unit"])
                .pipe(lambda df_:pd.pivot(df_,values="actor_freq",columns="variable",index="id"))
                .reset_index()
            )
    
def tweak_director(df_all_casting,df):
    return (#2- extracting director_weights
                    df_all_casting 
                    .melt(id_vars="id",value_vars=["director_name"])
                    .sort_values(by=["id","variable"])
                    .merge(df[["id","year"]],on="id",how="left")
                    .replace("NaN",np.NaN)
                    .replace("none",np.NaN)
                    .dropna()
                    .assign(unit=1)
                    .sort_values(by=["value","year"])
                    .assign(director_freq=lambda df_:df_.groupby("value").cumsum()["unit"])
                    .drop(columns=["variable","value","year","unit"])
                    .merge(df_all_casting[["id","director_name"]],on="id",how="left")
    )
      
def data_awards_cleaned_func(df_awards):
    return (
        df_awards
                .assign(outcome=lambda df_:df_["outcome"].replace(["2nd place", "3rd place"], "Won"))
                .groupby(["director_name", "year","outcome"])
                .count()
                .unstack()
                .fillna(0)
                .reset_index()
                .iloc[:,0:4]
                .pipe(lambda df_:pd.DataFrame(df_.values,columns=["director_name","year","nominated","won"]))
                .assign(nominated_cumsum = lambda df_: df_.groupby("director_name")["nominated"].transform(pd.Series.cumsum),
                    won_cumsum = lambda df_: df_.groupby("director_name")["won"].transform(pd.Series.cumsum),
                    )
                .set_index(["director_name","year"])
            )
    
def tweak_data(df_all_casting,df_all_details,df_awards):
    df=tweak_details(df_all_details)
    actors_df=tweak_cast(df_all_casting,df)
    director_df=tweak_director(df_all_casting,df)
    
    data_awards_cleaned=data_awards_cleaned_func(df_awards)
    df=(
    df
    .merge(actors_df,on=['id'],how="left")
    .merge(director_df,on=['id'],how="left")
    # .pipe(get_var, 'new_cols') 
    .drop(columns=["id","genres","spoken_languages_number"] )
    ).assign(
        fame_nominated= lambda df_:df_.apply(lambda df_:fame_func(data_awards_cleaned,df_.director_name,df_.year),axis=1),
        fame_won= lambda df_:df_.apply(lambda df_:fame_func(data_awards_cleaned,df_.director_name,df_.year,stat=1),axis=1)
        
    ).drop(columns=["director_name","year"] )


    df_nulls=df.replace(-999,np.NAN)
    df
    
    return df,df_nulls

def tweak_data4_prediction(df_all_casting,record_casting,df_all_details,record_data,df_awards):
    record_data.iloc[0,0]=-1001
    record_data.reset_index(inplace=True,drop=True)
    record_casting.iloc[0,0]=-1001
    record_casting.reset_index(inplace=True,drop=True)
    df_all_casting=pd.concat([df_all_casting.copy(deep=True),record_casting])
    df_all_details=pd.concat([df_all_details.copy(deep=True),record_data])

    df=tweak_details(df_all_details)

    actors_df=tweak_cast(df_all_casting,df)

    director_df=tweak_director(df_all_casting,df)

    data_awards_cleaned=data_awards_cleaned_func(df_awards)
    df=(
    df
    .merge(actors_df,on=['id'],how="left")
    .merge(director_df,on=['id'],how="left")
    .drop(columns=[
        # "id",
        "genres","spoken_languages_number"] )
    ).assign(
        fame_nominated= lambda df_:df_.apply(lambda df_:fame_func(data_awards_cleaned,df_.director_name,df_.year),axis=1),
        fame_won= lambda df_:df_.apply(lambda df_:fame_func(data_awards_cleaned,df_.director_name,df_.year,stat=1),axis=1)
        
    ).drop(columns=["director_name","year"] )

    
    
    
    return df.query("id==-1001").drop(columns=["id","revenue","popularity"])

In [226]:
cat_transformer = Pipeline([
    ("ohc",OneHotEncoder(handle_unknown='ignore'))
    ])
preprocessor = ColumnTransformer([
    ('cat_tr', cat_transformer, ['day_of_week']),
    ('cat_imputer', SimpleImputer(strategy="most_frequent"), ['day_of_week','actor1_name', 'actor2_name', 
                                                            'actor3_name', 'actor4_name', 'actor5_name', 
                                                            'director_freq', 'fame_nominated', 'fame_won']),
    ('imputer',SimpleImputer(strategy="median"),["runtime",'age', 'sin_month', 'cos_month'])

    ],remainder="passthrough")

estimator=Pipeline(
    [
        ("estimator",XGBRegressor(
            max_depth=6,           # maximum depth of each tree - try 2 to 10
            learning_rate=0.01,    # effect of each tree - try 0.0001 to 0.1
            n_estimators=1000,     # number of trees (that is, boosting rounds) - try 1000 to 8000
            min_child_weight=1,    # minimum number of houses in a leaf - try 1 to 10
            colsample_bytree=0.7,  # fraction of features (columns) per tree - try 0.2 to 1.0
            subsample=0.7,         # fraction of instances (rows) per tree - try 0.2 to 1.0
            reg_alpha=0.5,         # L1 regularization (like LASSO) - try 0.0 to 10.0
            reg_lambda=1.0,        # L2 regularization (like Ridge) - try 0.0 to 10.0
            num_parallel_tree=1,   # set > 1 for boosted random forests
        ))
        # ("estimator",lgbm.LGBMRegressor())
        # ("estimator",random_search.best_estimator_)
    ]
)

m=Pipeline([
        ('preprocessor',preprocessor),
        # ('imputer',KNNImputer()),
        ('scaler',StandardScaler()),
        ("estimator",estimator)
        
    ])
m

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  Pipeline(steps=[('ohc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['day_of_week']),
                                                 ('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['day_of_week', 'actor1_name',
                                                   'actor2_name', 'actor3_name',
                                                   'actor4_name', 'actor5_name',
                                                   'director_freq',
                                                   'fame_nominate...
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.01,
                                               max_delta_step=None, max_depth=6,
                                               min_child_weight=1, missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=1000, n_jobs=None,
                                               num_parallel_tree=1,
                                               predictor=None,
                                               random_state=None, reg_alpha=0.5,
                                               reg_lambda=1.0,
                                               scale_pos_weight=None,
                                               subsample=0.7, tree_method=None,
                                               validate_parameters=None,
                                               verbosity=None))]))])

In [227]:
__,df_null=tweak_data(df_all_casting,df_all_details,df_awards)

/home/ashamsa/.pyenv/versions/lewagon/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [228]:
X=df_null.drop(columns=["revenue","popularity"])
# X=df.drop(columns=["revenue","popularity"])
y=df_null[["revenue","popularity"]]
target_1=y.iloc[:,0]
target_2=y.iloc[:,1]

In [229]:
kf=KFold(n_splits=5,random_state=1,shuffle=True)


In [232]:
scores=cross_val_score(m, X, y.iloc[:,0], cv=kf)
scores

array([0.61612038, 0.66027439, 0.59819741, 0.63787776, 0.57851687])

In [233]:
scores.mean()

0.6181973626748498

In [234]:
#sample record format for prediction
id=597
record_data=df_all_details.query("id==@id")
record_casting=df_all_casting.query("id==@id")
record_data

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
474,597,200000000,Drama|Romance|Thriller,tt0120338,en,Titanic,"84 years later, a 101-year-old woman named Ros...",10.517521,Paramount Pictures,United States of America,18/11/1997,1845034188,194.0,English,Released,Nothing on Earth could come between them.,Titanic,7.5,6844,3,1,6


In [235]:
record_casting

,id,actor1_name,actor1_gender,actor2_name,actor2_gender,actor3_name,actor3_gender,actor4_name,actor4_gender,actor5_name,actor5_gender,actor_number,director_name,director_gender,director_number,producer_name,producer_number,screeplay_name,editor_name
474,597,Kate Winslet,1,Leonardo DiCaprio,2.0,Frances Fisher,1,Billy Zane,2,Kathy Bates,1,136,James Cameron,2.0,1,James Cameron,2,James Cameron,Conrad Buff IV


In [236]:
X.shape

(8657, 42)

In [237]:
df_all_casting.shape

(329044, 19)

In [238]:
df_all_details.shape

(329044, 22)

In [239]:
xx=tweak_data4_prediction(df_all_casting,record_casting,df_all_details,record_data,df_awards)


/home/ashamsa/.pyenv/versions/lewagon/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [241]:
xx.shape

(1, 42)

In [242]:
m

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  Pipeline(steps=[('ohc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['day_of_week']),
                                                 ('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['day_of_week', 'actor1_name',
                                                   'actor2_name', 'actor3_name',
                                                   'actor4_name', 'actor5_name',
                                                   'director_freq',
                                                   'fame_nominate...
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.01,
                                               max_delta_step=None, max_depth=6,
                                               min_child_weight=1, missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=1000, n_jobs=None,
                                               num_parallel_tree=1,
                                               predictor=None,
                                               random_state=None, reg_alpha=0.5,
                                               reg_lambda=1.0,
                                               scale_pos_weight=None,
                                               subsample=0.7, tree_method=None,
                                               validate_parameters=None,
                                               verbosity=None))]))])

In [243]:
m.fit(X,y.iloc[:,0])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  Pipeline(steps=[('ohc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['day_of_week']),
                                                 ('cat_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['day_of_week', 'actor1_name',
                                                   'actor2_name', 'actor3_name',
                                                   'actor4_name', 'actor5_name',
                                                   'director_freq',
                                                   'fame_nominate...
                                               importance_type=None,
                                               interaction_constraints='',
                                               learning_rate=0.01,
                                               max_delta_step=0, max_depth=6,
                                               min_child_weight=1, missing=nan,
                                               monotone_constraints='()',
                                               n_estimators=1000, n_jobs=12,
                                               num_parallel_tree=1,
                                               predictor='auto', random_state=0,
                                               reg_alpha=0.5, reg_lambda=1.0,
                                               scale_pos_weight=1,
                                               subsample=0.7,
                                               tree_method='exact',
                                               validate_parameters=1,
                                               verbosity=None))]))])

In [244]:
m.predict(xx)

array([1.7235707e+09], dtype=float32)

In [133]:
m.predict(xx)

array([1.219621e+09], dtype=float32)

In [134]:
yy

,revenue,popularity
0,1845034188,10.517521


In [177]:
record_data

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,597,200000000,Drama|Romance|Thriller,tt0120338,en,Titanic,"84 years later, a 101-year-old woman named Ros...",10.517521,Paramount Pictures,United States of America,18/11/1997,1845034188,194.0,English,Released,Nothing on Earth could come between them.,Titanic,7.5,6844,3,1,6
